In [79]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from crop_image import *
import imutils
from imutils import contours as imcnts
import skimage.io as io
%matplotlib inline

In [80]:
image = cv2.imread('omr.png')
cv2.imshow('image.jpg',image)
cv2.waitKey(0)
number_of_choices=4
number_of_columns=2

In [81]:
gray_scale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
cv2.imshow('gray scale image.jpg',gray_scale)
cv2.waitKey(0)

-1

In [82]:
"""gaussian = cv2.GaussianBlur(image,(3,3),0)
cv2.imshow('image',gaussian)
cv2.waitKey(0)"""

"gaussian = cv2.GaussianBlur(image,(3,3),0)\ncv2.imshow('image',gaussian)\ncv2.waitKey(0)"

In [83]:
edged = cv2.Canny(gray_scale,50,200)
# cv2.imshow('edged.jpg',edged)
# cv2.waitKey(0)

In [84]:
contours,_= cv2.findContours(edged, 
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

print("Number of Contours found = " + str(len(contours)))
  
cv2.drawContours(image, contours, -1,(0,255,0), 3)
  
# cv2.imshow('Contours.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

Number of Contours found = 4


In [85]:
#sorting all contours
if(len(contours)>0):
    contours=sorted(contours,key=cv2.contourArea,reverse=True)

In [86]:
cv2.drawContours(image, contours, 0,(0,255,0), 3)
  
# cv2.imshow('Contours3.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

In [87]:
paper_contour=None
for cnt in contours:
    peri = cv2.arcLength(cnt,True)
    print(peri)
    # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
    ratio=0.02
    approx=cv2.approxPolyDP(cnt,ratio*peri,True)
    cv2.drawContours(image, [approx], 0,(0,0,255), 3)
    print(len(approx))
    coordinates=approx.ravel()
    for i in range(0,len(coordinates),2):
        x=coordinates[i]
        y=coordinates[i+1]
        cv2.putText(image, "vertex"+str(i//2), (x, y),cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 0)) 
    if(len(approx)==4):
        paper_contour=approx
        break
# cv2.imshow('Contours2.jpg', image)
# cv2.waitKey(0)
image = cv2.imread('omr.png')

1730.0
4


In [88]:
print(paper_contour.shape)

(4, 1, 2)


In [89]:
paper= four_point_transform(image, paper_contour.reshape(4, 2))  
# cv2.imshow('paper', paper)
# cv2.waitKey(0)

In [90]:
gray_scale_paper= four_point_transform(gray_scale, paper_contour.reshape(4, 2))  
# cv2.imshow('paper', gray_scale_paper)
# cv2.waitKey(0)

In [91]:
# _,thresholded=cv2.threshold(gray_scale_paper,230,255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
# cv2.imshow('paper1', thresholded)
# cv2.waitKey(0)

In [92]:
_,thresholded=cv2.threshold(gray_scale_paper,230,255, cv2.THRESH_BINARY_INV)
cv2.imshow('paper2', thresholded)
cv2.waitKey(0)

-1

In [93]:
pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [94]:
def test_cnts(cnts):
    cv2.drawContours(thresholded, cnts, -1,255, 3)
  
    cv2.imshow('Contours.jpg', thresholded)
    cv2.waitKey(0)

In [108]:
pre_question_cnts=[]
value=(15,50)
i=0
for cnt in pap_cnts:
    (x,y,w,h)=cv2.boundingRect(cnt)
    aspect_ratio=w/h
    if(w>=value[0] and w<=value[1] and h>=value[0] and h<=value[1] and aspect_ratio>=0.9 and aspect_ratio<=1.1):
        pre_question_cnts.append(cnt)

#1 368 325
#2 342 325
#3 315 325
#4 288 325
#5 369 296
#6 342 296
#7 314 296
#8 287 296
#9 174 296
#10 147 296
#11 119 296
#12 92 296
#13 369 266
#14 342 266
#15 314 266
#16 287 266
#17 174 266
#18 147 266
#19 119 266
#20 92 266
#21 369 236
#22 342 236
#23 314 236
#24 287 236
#25 174 236
#26 147 236
#27 119 236
#28 92 236
#29 369 206
#30 342 206
#31 314 206
#32 287 206
#33 174 206
#34 147 206
#35 119 206
#36 92 206
#37 371 129
#38 343 129
#39 316 129
#40 288 129
#41 174 129
#42 146 129
#43 119 129
#44 91 129
#45 371 99
#46 343 99
#47 316 99
#48 288 99
#49 174 99
#50 146 99
#51 119 99
#52 91 99
#53 371 69
#54 343 69
#55 316 69
#56 288 69
#57 174 69
#58 146 69
#59 119 69
#60 91 69
#61 371 39
#62 343 39
#63 316 39
#64 288 39
#65 174 39
#66 146 39
#67 119 39
#68 91 39


In [121]:
print(len(pre_question_cnts),len(pap_cnts))

68 90


In [135]:
question_cnts,_=imcnts.sort_contours(pre_question_cnts,method='top-to-bottom')
xs_set=set()
i=0

for cnt in question_cnts:
    (x,y,w,h)=cv2.boundingRect(cnt)
    if(x not in xs_set and all([ x-i not in xs_set and x+i not in xs_set for i in np.arange(1,5)])):
        xs_set.add(x)
#     if(i<8):
#         print(f'#{i+1}',x,y)
#     i+=1
number_of_bubbles=len(xs_set)
#print(xs_set,len(xs_set))
xs=np.array(list(sorted(xs_set)))
dist=np.append(xs[1:],xs[-1])-xs
avg=np.mean(dist)
number_of_choices=number_of_bubbles//number_of_columns
number_of_columns=np.sum(dist>avg)+1
#print(np.sum(dist>avg))
#print(list(sorted(xs_set)))
#1 371 39
#2 343 39
#3 316 39
#4 288 39
#5 174 39
#6 146 39
#7 119 39
#8 91 39

#1 371 39
#2 343 39
#3 316 39
#4 288 39
#5 174 39
#6 146 39
#7 119 39
#8 91 39
{288, 174, 146, 371, 119, 343, 91, 316} 8
1
[91, 119, 146, 174, 288, 316, 343, 371]


In [128]:
tt=[i>2 for i in np.arange(1,5)]
print(all(tt))

False


In [120]:
# Answer_key= {0:1, 1:4, 2:0, 3:2}
# correct=0
# flag=False
# cv2.drawContours(paper,question_cnts,0,(255,0,0), 1)
# cv2.imshow('hgsvrgvj',paper)
# cv2.waitKey(0)
# cnt=question_cnts[0]
# peri = cv2.arcLength(cnt,True)
# #print(peri)
# # to approximate the given shape to polygon (high value leads to underestimation and low value leads to overestimation)
# ratio=0.00005
# approx=cv2.approxPolyDP(cnt,ratio*peri,True)
# (x,y,w,h)=cv2.boundingRect(cnt)
# aspect_ratio=w/h
# print(len(approx))
# paper= four_point_transform(image, paper_contour.reshape(4, 2))  

In [99]:
print(len(question_cnts))

68


In [100]:
#test_cnts(question_cnts)

In [101]:
# _,thresholded=cv2.threshold(gray_scale_paper,150,255, cv2.THRESH_BINARY_INV)
# pap_cnts,_=cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [102]:
colors=[(255,0,0)
,(0,255,0)
,(0,0,255)
,(0,255,255)
,(255,255,0)
,(0,0,255)
,(0,255,255)
,(255,0,255)
,(192,192,192)
,(128,128,128)
,(128,0,0)
,(128,128,0)
,(0,128,0)
,(128,0,128)
,(0,128,128)
,(0,0,128),
       (255,215,0)]

In [103]:
counter=0
qs=0
for (row,i) in enumerate(np.arange(0,len(question_cnts),number_of_columns*number_of_choices)):

    curr_row_cnts,_=imcnts.sort_contours(question_cnts[i:i+number_of_columns*number_of_choices])
    for k in np.arange(0,len(curr_row_cnts),number_of_choices):
        curr_ques_cnts=curr_row_cnts[k:k+number_of_choices]
        color1=colors[qs%len(colors)]
        cv2.drawContours(paper,curr_ques_cnts,-1,color1, 1)
        bubbled=None
        for (j,c) in enumerate(curr_ques_cnts):
            mask = np.zeros(thresholded.shape, dtype="uint8")
            #negative one at the last arguement to fill the contours
            cv2.drawContours(mask, [c], -1, 255, -1)
            mask= cv2.bitwise_and(thresholded, mask)
            total= cv2.countNonZero(mask)

            if bubbled is None or total >bubbled[0]:
                bubbled= (total, j)
            color= (0, 0, 255)
        print(f"qestion#{qs+1}: {chr(bubbled[1]+ord('A'))}")
        qs+=1
    counter+=1
print(counter)
counter=0
cv2.imshow('paper', paper)
cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

qestion#1: A
qestion#2: A
qestion#3: C
qestion#4: C
qestion#5: A
qestion#6: D
qestion#7: D
qestion#8: C
qestion#9: D
qestion#10: B
qestion#11: C
qestion#12: C
qestion#13: C
qestion#14: B
qestion#15: C
qestion#16: C
qestion#17: D
9


In [104]:
correct=0
print(correct

SyntaxError: incomplete input (3508751463.py, line 2)

In [ ]:
# cv2.imshow('Contours.jpg', thresholded)
# cv2.waitKey(0)

In [ ]:
for i,cnt in enumerate(question_cnts):
    color1=(255,0,0) if i%2==0 else (0,0,255)
    cv2.drawContours(paper,[cnt],-1,color1, 1)
# cv2.imshow('paper', paper)
# cv2.waitKey(0)
paper= four_point_transform(image, paper_contour.reshape(4, 2)) 

In [ ]:
def show_images(images,titles=None):
    #This function is used to show image(s) with titles by sending an array of images and an array of associated titles.
    # images[0] will be drawn with the title titles[0] if exists
    # You aren't required to understand this function, use it as-is.
    n_ims = len(images)
    if titles is None: titles = ['(%d)' % i for i in range(1,n_ims + 1)]
    fig = plt.figure()
    n = 1
    for image,title in zip(images,titles):
        a = fig.add_subplot(1,n_ims,n)
        if image.ndim == 2: 
            plt.gray()
        plt.imshow(image)
        a.set_title(title)
        n += 1
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_ims)
    plt.show()



In [ ]:
print(contours)

In [ ]:
f"adham{chr(bubbled[1]+ord('A'))}"